In [73]:
import pandas as pd
import numpy as np
import ast

fixedDf = pd.read_csv('data/fixedData.csv')
fixedDf.head()

,Unnamed: 0,name,address,coordinates,workday_timing,closed_on,reviews,rating,most_popular_times,popular_times,reviewer_name,rating_review,review_text,accessibility_enabled,children_enabled,types,city
0,0,Agro Tainment Wisata Kampung Markisa,"Agro Tainment Wisata Kampung Markisa, Blunyahr...","-7.774189,110.3647986",Not Present,Buka Setiap Hari,2.0,5.0,"07.00, 08.00, 09.00\n---\nLabel Waktu: 07.00\n...",Senin:\n 06.00: 0% | Diam\n 07.00: 25% |...,"['Deyze Nghokeh', 'Khoari Koutsar', 'Deyze Ngh...","[5.0, 5.0, 5.0, 5.0]","['Teks review tidak tersedia', 'Teks review ti...","Pintu masuk khusus pengguna kursi roda, Tempat...",Cocok untuk anak-anak,NaN,"Kota Yogyakarta, Daerah Istimewa Yogyakarta"
1,1,Ancol Wisata Alam,"Ancol Wisata Alam, Unnamed Road, Selingan, Kar...","-7.6668655999999995,110.266713",Not Present,Buka Setiap Hari,390.0,4.4,Tidak Tersedia,Tidak Tersedia,"['Ifandri Dwi Aryono (IFANDRI DWI ARYONO)', 'C...","[5.0, 5.0, 4.0, 5.0, 4.0, 2.0, 5.0, 4.0]",['salah catu calon destinasi wisata alam berik...,"Pintu masuk khusus pengguna kursi roda, Tempat...",Cocok untuk anak-anak,NaN,"Kabupaten Magelang, Jawa Tengah"
2,2,BANYU WIYOSO DAGEN (Kolam Ikan),"4846+WH5 BANYU WIYOSO DAGEN (Kolam Ikan), Gumu...","-7.8927344999999995,110.3114727",Not Present,Buka Setiap Hari,4.0,5.0,Tidak Tersedia,Tidak Tersedia,"['Laili Isna Fatkhurrahmah', 'Dian Kusuma Dewi...","[5.0, 5.0, 5.0, 5.0]",['Tempat yang asik buat jalan2 pagi atau sore....,"Pintu masuk khusus pengguna kursi roda, Tempat...",Cocok untuk anak-anak,NaN,"Kabupaten Bantul, Daerah Istimewa Yogyakarta"
3,3,Bendhung Lepen,"Bendhung Lepen, Mrican UH7/ 338, Giwangan, Kec...","-7.8316668,110.3945119",Not Present,Buka Setiap Hari,1336.0,4.7,"17.00, 16.00, 18.00\n---\nLabel Waktu: 17.00\n...",Senin:\n 04.00: 0% | Diam\n 05.00: 6% | ...,"['Muhammad Hidayatullah', 'suwarso doang', 'L ...","[5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 5.0, 5.0]","['tempat menyenangkan untuk rekreasi, bisa dud...","Pintu masuk khusus pengguna kursi roda, Tempat...","Cocok untuk anak-anak, Cocok untuk ulang tahun...",NaN,"Kota Yogyakarta, Daerah Istimewa Yogyakarta"
4,4,Bendungan Irigasi Jalan Samas,"278H+G39 Bendungan Irigasi Jalan Samas, Unname...","-7.9837082,110.2777382",Not Present,Buka Setiap Hari,1.0,5.0,Tidak Tersedia,Tidak Tersedia,['Bastian Viery'],[5.0],['Bagus'],Tempat parkir khusus pengguna kursi roda,Tidak cocok untuk anak-anak,NaN,"Kabupaten Bantul, Daerah Istimewa Yogyakarta"


# Get Data Reviewer

In [74]:
fixedDf = fixedDf.rename(columns={'Unnamed: 0': 'place_id'})
fixedDf.place_id.nunique()
fixedDf.to_csv('data/fixedData.csv', index=False)

## Tambah Kolom Place Id

In [75]:
reviewerDf = fixedDf[['place_id','reviewer_name', 'rating_review', 'review_text']]
reviewerDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 455 entries, 0 to 454
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   place_id       455 non-null    int64 
 1   reviewer_name  409 non-null    object
 2   rating_review  409 non-null    object
 3   review_text    402 non-null    object
dtypes: int64(1), object(3)
memory usage: 14.3+ KB


## Memisahkan Data List dalam Bentuk Per Baris Data

### Mengisi Nilai Null dengan Anonim

In [76]:
# Langkah 1: Mengisi nilai null dengan string '["anonim"]'
reviewerDf.loc[:, 'reviewer_name'] = reviewerDf['reviewer_name'].fillna('["unknown"]')
reviewerDf.loc[:, 'rating_review'] = reviewerDf['rating_review'].fillna('["0.0"]')
reviewerDf.loc[:, 'review_text'] = reviewerDf['review_text'].fillna('["unknown"]')

### Preprocess Bentuk List

In [77]:
# Fungsi aman untuk menggunakan ast.literal_eval
def safe_literal_eval(val):
    try:
        if isinstance(val, str):
            return ast.literal_eval(val)  # Ubah hanya jika val adalah string
        return val
    except (ValueError, SyntaxError):
        return val  # Kembalikan val asli jika tidak bisa diuba

In [78]:
# Langkah 2: Mengonversi string yang terlihat seperti list menjadi list Python menggunakan ast.literal_eval secara aman
reviewerDf.loc[:, 'reviewer_name'] = reviewerDf['reviewer_name'].apply(safe_literal_eval)
reviewerDf.loc[:, 'rating_review'] = reviewerDf['rating_review'].apply(safe_literal_eval)
reviewerDf.loc[:, 'review_text'] = reviewerDf['review_text'].apply(safe_literal_eval)

### Pemecahan List Jadi Per Baris

In [79]:
# Langkah 3: Menentukan panjang maksimum list dalam setiap kolom
def get_max_len(col):
    return col.apply(lambda x: len(x) if isinstance(x, list) else 0).max()

max_len_reviewer = get_max_len(reviewerDf['reviewer_name'])
max_len_rating = get_max_len(reviewerDf['rating_review'])
max_len_review = get_max_len(reviewerDf['review_text'])
max_len = max(max_len_reviewer, max_len_rating, max_len_review)

In [80]:
# Langkah 4: Menyamakan panjang list di semua kolom dengan mengisi NaN pada list yang lebih pendek
def pad_list_with_nan(lst, max_len):
    if isinstance(lst, list):
        return lst + [np.nan] * (max_len - len(lst))
    else:
        return [np.nan] * max_len  # Jika tidak ada list, buat list dengan NaN
reviewerDf.loc[:, 'reviewer_name'] = reviewerDf['reviewer_name'].apply(lambda x: pad_list_with_nan(x, max_len))
reviewerDf.loc[:, 'rating_review'] = reviewerDf['rating_review'].apply(lambda x: pad_list_with_nan(x, max_len))
reviewerDf.loc[:, 'review_text'] = reviewerDf['review_text'].apply(lambda x: pad_list_with_nan(x, max_len))

# Langkah 5: Meledakkan (explode) kolom reviewer_name, rating_review, dan review_text
reviewerDf = reviewerDf.explode(['reviewer_name', 'rating_review', 'review_text']).reset_index(drop=True)

### Hapus Data NaN (Dummy)

In [81]:
reviewerDf = reviewerDf.dropna(axis=0)

### Tambah Data User ID

In [82]:
reviewerDf['user_id'] = pd.factorize(reviewerDf['reviewer_name'])[0] + 1
# Ubah Urutan Kolom Agar Mudah dibaca
reviewerDf = reviewerDf[['place_id', 'user_id', 'reviewer_name', 'rating_review', 'review_text']]
reviewerDf.head(20)

,place_id,user_id,reviewer_name,rating_review,review_text
0,0,1,Deyze Nghokeh,5.0,Teks review tidak tersedia
1,0,2,Khoari Koutsar,5.0,Teks review tidak tersedia
2,0,1,Deyze Nghokeh,5.0,Teks review tidak tersedia
3,0,2,Khoari Koutsar,5.0,Teks review tidak tersedia
32,1,3,Ifandri Dwi Aryono (IFANDRI DWI ARYONO),5.0,salah catu calon destinasi wisata alam berikut...
33,1,4,Canesiase Marshallenn,5.0,"Dalam tahap pembangunan, tapi bisa untuk duduk..."
34,1,5,Haryadi,4.0,Belum jadi saja cakep apalagi kalau nanti suda...
35,1,6,Daniel Dicky,5.0,"Wisata alam di tepi Kali Progo, suasananya asr..."
36,1,7,Kris Mheilda Setiawati (Ida Mandalawangi),4.0,"Di sini lokasi hulu Selokan Mataram, Selokan V..."
37,1,8,MRTN,2.0,Salah satu spot Foto dan juga gowes atau pun b...


In [83]:
reviewerDf.to_csv('data/data-user/originalData.csv', index=False)

# Exploratory Data Analysis (EDA)

Pertanyaan:
1. Berapa user id yang lebih dari 1 nilai unik yang ada di dalam data?
2. place_id apa saja yang direview oleh user id yang tersaring di pertanyaan 1?

## Jawab No 1

In [84]:
# Menghitung jumlah kemunculan setiap user_id yang muncul lebih dari 1 kali
user_id_counts = reviewerDf['user_id'].value_counts()
filtered_user_ids = user_id_counts[user_id_counts > 1]

# Visualisasikan hasil group user_id yang muncul lebih dari 1 kali dengan countnya
cntByUsrID = pd.DataFrame({'user_id': filtered_user_ids.index, 'count': filtered_user_ids.values})
# Mengubah hasil menjadi format yang lebih mudah dibaca
cntByUsrID = cntByUsrID.groupby('count')['user_id'].apply(lambda x: ', '.join(x.astype(str))).reset_index()
cntByUsrID

,count,user_id
0,2,"865, 50, 51, 52, 53, 47, 372, 365, 353, 354, 3..."
1,3,"122, 336, 335, 334, 333, 508, 332, 739, 745, 8..."
2,4,"661, 106, 110, 604, 598, 599, 603, 600, 658, 6..."
3,5,"185, 743, 430, 132"
4,6,"352, 59, 361"
5,7,49
6,46,902


In [85]:
reviewerDf.loc[reviewerDf['user_id'] == 49, 'reviewer_name']

291     wahyu riyadi Herjito
299     wahyu riyadi Herjito
321     wahyu riyadi Herjito
329     wahyu riyadi Herjito
337     wahyu riyadi Herjito
1089    wahyu riyadi Herjito
1097    wahyu riyadi Herjito
Name: reviewer_name, dtype: object

In [86]:
cntByUsrID.to_csv('data/data-user/countUserPerReview.csv', index=False)

## Jawab No 2

In [87]:
# All
# Memodifikasi kode untuk memisahkan review_text dengan newline (\n) tanpa menggunakan unique
grpPlaceAll = reviewerDf.groupby('user_id').apply(
    lambda x: pd.DataFrame({
        'place_ids': [', '.join(x['place_id'].astype(str).unique())],
        'ratings': [', '.join(x['rating_review'].astype(str))],
        'reviews': ['\n'.join(x['review_text'])]  # Memisahkan review dengan \n
    })
).reset_index(level=1, drop=True).reset_index()

grpPlaceAll

C:\Users\legion\AppData\Local\Temp\ipykernel_13800\2054939666.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grpPlaceAll = reviewerDf.groupby('user_id').apply(


,user_id,place_ids,ratings,reviews
0,1,0,"5.0, 5.0",Teks review tidak tersedia\nTeks review tidak ...
1,2,0,"5.0, 5.0",Teks review tidak tersedia\nTeks review tidak ...
2,3,1,5.0,salah catu calon destinasi wisata alam berikut...
3,4,1,5.0,"Dalam tahap pembangunan, tapi bisa untuk duduk..."
4,5,1,4.0,Belum jadi saja cakep apalagi kalau nanti suda...
...,...,...,...,...
982,983,427,5.0,"Tempatnya syahdu, ada angkringan di pinggir ka..."
983,984,430,5.0,"enak dan harganya murah,cocok untuk sarapan at..."
984,985,433,5.0,Salah satu destinasi Wisata kekinian. Disambut...
985,986,435,5.0,"Sanggarnya bersih dan rapih, barang-barangnya ..."


In [88]:
# Sebagian
# Filter dataframe berdasarkan user_id yang muncul lebih dari 1 kali
filtered_df = reviewerDf[reviewerDf['user_id'].isin(filtered_user_ids.index)]

# Modifikasi kode untuk mengelompokkan data berdasarkan user_id yang sudah difilter
grpUserFiltered = filtered_df.groupby('user_id').apply(
    lambda x: pd.DataFrame({
        'place_ids': [', '.join(x['place_id'].astype(str).unique())],
        'ratings': [', '.join(x['rating_review'].astype(str))],
        'reviews': ['\n'.join(x['review_text'])]  # Memisahkan review dengan \n
    })
).reset_index(level=1, drop=True).reset_index()
grpUserFiltered

C:\Users\legion\AppData\Local\Temp\ipykernel_13800\2304598670.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grpUserFiltered = filtered_df.groupby('user_id').apply(


,user_id,place_ids,ratings,reviews
0,1,0,"5.0, 5.0",Teks review tidak tersedia\nTeks review tidak ...
1,2,0,"5.0, 5.0",Teks review tidak tersedia\nTeks review tidak ...
2,8,"1, 63","2.0, 5.0, 5.0",Salah satu spot Foto dan juga gowes atau pun b...
3,16,"3, 254, 281","5.0, 5.0, 5.0",Wisata murah meriah asri dan alami dengan sent...
4,21,"3, 123","5.0, 4.0",Tempat ini semacam wisata kali namun berisi ik...
...,...,...,...,...
364,894,136,"5.0, 5.0","Bagus, cocok untuk keluarga\nBagus, cocok untu..."
365,895,136,"5.0, 5.0",tempat wisata keluarga dan anak anak. bisa ber...
366,896,136,"4.0, 4.0","Tempatnya bagus bersih dan nyaman,hanya petuga..."
367,902,"150, 159, 168, 184, 189, 200, 210, 211, 216, 2...","0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...",unknown\nunknown\nunknown\nunknown\nunknown\nu...


In [89]:
grpUserFiltered.to_csv('data/data-user/countUserPerUniqueReview.csv', index=False)

In [90]:
# Ambil data reviews dari user_id 16
reviews = grpUserFiltered.loc[grpUserFiltered['user_id'] == 16, 'reviews'].values

# Jika reviews ditemukan, pisahkan berdasarkan '\n' dan print setiap review
if reviews:
    for review in reviews[0].split('\n'):
        print(review)
else:
    print("No reviews found for user_id 16.")

Wisata murah meriah asri dan alami dengan sentuhan seni dan teknologi tata lingkungan. Ramai, terutama di akhir pekan. Tiket masuk gratis-tis, hanya dikenakan bea parkir, itupun sukarela. Anda akan mendapatkan pemandangan dan suasana kali yang ditata rapi dan indah dengan cat warna-warni serta akses tangga yang tapi. Area ini menyatu antara kali/selokan aneka ikan dan bendungan kali Gajah Wong, sungai yang membelah kota Jogja, dekat dengan terminal Giwangan. Aneka kuliner dan makanan kecil banyak tersedia di lapak2 dan warung di sekitar kali. Di waktu-waktu tertentu ada kunjungan perpustakaan keliling yang buka disini. Cocok untuk wisata keluarga di tengah penatnya keramaian kehidupan kota
Belum buka resmi, cuman bisa lihat dari parkir/area kuliner. Cukup bayar parkir, sekalian mampir sini sebelum melanjutkan perjalanan
Jika cuaca cerah bisa menikmati area sungai yg dihiasi hamparan batu-batu, terlebih di musim kemarau, sangat jelas hingga dasar sungai mengingat debit air sunganyai kec